In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import random

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
data = r"D:\users\new owner\Desktop\TKS\Christmas Break\gtsrb-german-traffic-sign"
img_size = 32

In [ ]:
def load_data(dataset):
    rows = pd.read_csv(dataset)
    rows = rows.sample(frac=1).reset_index(drop=True)
    images = []
    classes = []
    for i, row in rows.iterrows():
        img_class = row["ClassId"]
        img_path = row["Path"]

        image = os.path.join(data, img_path)

        image = cv2.imread(image)
        image_rs = cv2.resize(image, (img_size, img_size), 3)

        R, G, B = cv2.split(image_rs)

        img_r = cv2.equalizeHist(R)
        img_g = cv2.equalizeHist(G)
        img_b = cv2.equalizeHist(B)

        new_image = cv2.merge((img_r, img_g, img_b))

        images.append(new_image)
        classes.append(img_class)

        if i % 500 == 0:
            print(f"loaded: {i}")

    X = np.array(images)
    y = np.array(classes)
    
    return(X, y)

In [ ]:
train_data = r"D:\users\new owner\Desktop\TKS\Christmas Break\gtsrb-german-traffic-sign\Train.csv"
test_data = r"D:\users\new owner\Desktop\TKS\Christmas Break\gtsrb-german-traffic-sign\Test.csv"

In [ ]:
(trainX, trainY) = load_data(train_data)
(testX, testY) = load_data(test_data)

In [ ]:
train_X = trainX / 255.0
test_X = testX / 255.0

num_labels = len(np.unique(trainY))
train_Y = to_categorical(trainY, num_labels)
test_Y = to_categorical(trainY, num_labels)

class_totals = train_Y.sum(axis=0)
class_weight = class_totals.max() / class_totals

train_X[100].shape
train_X.shape

In [ ]:
class RoadSignClassifier:
    def createCNN(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        
        
        #1st convolution layer
        model.add(Conv2D(8, (5, 5), input_shape=inputShape, activation="relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(BatchNormalization())
        
        model.add(Conv2D(16, (3, 3), activation="relu"))
        model.add(BatchNormalization())
        model.add(Conv2D(16, (3, 3), activation="relu"))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
 
        model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
        model.add(BatchNormalization())
        model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        #fully connected layer
        model.add(Flatten())
        model.add(Dropout(0.5))
        model.add(Dense(128, activation="relu"))
        
        
        #output
        model.add(Dense(classes, activation="softmax"))
        return model
    
    

In [ ]:
data_aug = ImageDataGenerator(
rotation_range=10,
zoom_range=0.15,
width_shift_range=0.1,
height_shift_range=0.1,
shear_range=0.15,
horizontal_flip=False,
vertical_flip=False)


epochs = 10
batch_size = 64
learning_rate = 0.001

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
pred_model = ModelCheckpoint('classifier.hdf5', save_best_only=True, monitor='accuracy')

In [ ]:
model = RoadSignClassifier.createCNN(width=img_size, height=img_size, depth=3, classes=43)
optimizer = Adam(lr=learning_rate, decay=learning_rate / (epochs))

model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])    


network = model.fit_generator(
    data_aug.flow(train_X, train_Y, batch_size=batch_size), 
    epochs=epochs,
    validation_data=(test_X, test_Y),
    class_weight=class_weight,
    verbose=1,
    callbacks=[pred_model])
    





In [ ]:
from tensorflow.keras.models import load_model
model_save = load_model("classifier.hdf5")


In [ ]:
def preprocess(image):    
    R, G, B = cv2.split(image)

    img_r = cv2.equalizeHist(R)
    img_g = cv2.equalizeHist(G)
    img_b = cv2.equalizeHist(B)

    new_image = cv2.merge((img_r, img_g, img_b))
    
    img = new_image/255
    
    return img

In [ ]:
img_size = 32
image = testX[685]
img = np.asarray(image)
img = cv2.resize(img, (img_size, img_size), 3)

In [ ]:
plt.imshow(img, cmap = ('gray'))

In [ ]:
img = preprocess(img)

In [ ]:
img = img.reshape(1, 32, 32, 3)

In [ ]:
print("predicted sign: "+ str(model_save.predict_classes(img)))